### Summer Days: wps_climdex_su

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) function, [climdex.su](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=25)

Takes a climdexInput object and computes the SU (summer days) climdexindex:  that is,  the annual count of days where daily maximum temperature exceeds 25 degreesCelsius.

In [ ]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url, local_path

In [ ]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [ ]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

In [ ]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [ ]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_su?

#### Run wps_climdex_su process with Rdata file

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="summer_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_su(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",
            output_path=output_file.name,
        )
url = output.get()[0]

#### Test output against expected output

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    urlretrieve(url, tmp_file.name)
    robjects.r("load(file='{}')".format(tmp_file.name))
    output_vector = robjects.r("summer_days")
    
robjects.r("load(file='{}')".format(resource_filename("tests","data/expected_summer_days.rda")))
expected_vector = robjects.r("expected_summer_days")
    
for index in range(len(expected_vector)):
    assert str(output_vector[index]) == str(expected_vector[index])